# The Cuppa3 Compiler

This is a notebook that let's us play around with the Cuppa3 compiler. Both as a whole running our testsuite and the individual parts.

In [1]:
from cuppa3_cc import cc
from exp2bytecode_interp import interp

## The test suite.

In [2]:
add = \
'''
declare add(a,b) 
{
    return a+b;
}

declare x = add(3,2);
put x;
'''

seqsum = \
'''
declare add(a,b) 
{
    return a+b;
}

declare seqsum(n) 
{
    
    declare i = 1;
    declare sum = 0;
        
    while (i <= n) 
    {
        sum = add(sum,i);
        i = i + 1;
    }
        
    put sum;
}

seqsum(10);
'''

and_prog = \
'''
declare and(a,b)
{
    return a*b;
}

put and(1,1);
put and(1,0);
put and(0,1);
put and(0,0);
'''

fact = \
'''
// computes the factorial of x = 3
declare x = 3;
declare y = 1;
while (1 <= x) 
{
      y = y * x;
      x = x - 1;
}
put y;
'''

factrec = \
'''
// recursive implementation of factorial
declare fact(x) 
{
     if (x <= 1)
        return 1;
     else 
        return fact(x-1) * x;
}

put fact(3);
'''

fold = \
'''
declare x = (3 + 2) / 5;
put x;
'''


func1 = \
'''
declare f () {
 put(1001);
}

f();
'''

In [22]:
interp(cc(add))

> 5


In [23]:
print(cc(seqsum))

	jump L28 ;
#  
# Start of function add
#  
add:
	pushf 3 ;
	store %tsx[0] %tsx[-4] ;
	store %tsx[-1] %tsx[-5] ;
	store %tsx[-2] (+ %tsx[0] %tsx[-1]) ;
	store %rvx %tsx[-2] ;
	popf 3 ;
	return ;
	popf 3 ;
	return ;
#  
# End of function add
#  
L28:
	noop ;
	jump L29 ;
#  
# Start of function seqsum
#  
seqsum:
	pushf 6 ;
	store %tsx[0] %tsx[-7] ;
	store %tsx[-1] 1 ;
	store %tsx[-2] 0 ;
L30:
	store %tsx[-3] (<= %tsx[-1] %tsx[0]) ;
	jumpF %tsx[-3] L31 ;
	pushv %tsx[-1] ;
	pushv %tsx[-2] ;
	call add ;
	popv ;
	popv ;
	store %tsx[-4] %rvx ;
	store %tsx[-2] %tsx[-4] ;
	store %tsx[-5] (+ %tsx[-1] 1) ;
	store %tsx[-1] %tsx[-5] ;
	jump L30 ;
L31:
	noop ;
	print %tsx[-2] ;
	popf 6 ;
	return ;
#  
# End of function seqsum
#  
L29:
	noop ;
	pushv 10 ;
	call seqsum ;
	popv ;
	stop ;



In [5]:
interp(cc(and_prog))

> 1
> 0
> 0
> 0


In [6]:
interp(cc(fact))

> 6


In [7]:
interp(cc(factrec))

> 6


In [8]:
interp(cc(fold))

> 1


In [9]:
interp(cc(func1))

> 1001


Testing our interpreter on the recursive computation of the Fibonacci sequence

In [10]:
fib = \
'''
declare fib(n)
{    
    if (n == 0)
        return 0;
    else if (n == 1)
        return 1;
    else
        return fib(n-1) + fib(n-2);
}

// fibonacci sequence: 0,1,1,2,3,5,8,13,21,34,55,89,144,...
declare i = 0
while (i <= 12)
{
    put fib(i)
    i = i + 1
}
'''

In [11]:
print(cc(fib))

	jump L12 ;
#  
# Start of function fib
#  
fib:
	pushf 8 ;
	store %tsx[0] %tsx[-9] ;
	store %tsx[-1] (== %tsx[0] 0) ;
	jumpF %tsx[-1] L13 ;
	store %rvx 0 ;
	popf 8 ;
	return ;
	jump L14 ;
L13:
	store %tsx[-2] (== %tsx[0] 1) ;
	jumpF %tsx[-2] L15 ;
	store %rvx 1 ;
	popf 8 ;
	return ;
	jump L16 ;
L15:
	store %tsx[-3] (- %tsx[0] 1) ;
	pushv %tsx[-3] ;
	call fib ;
	popv ;
	store %tsx[-4] %rvx ;
	store %tsx[-5] (- %tsx[0] 2) ;
	pushv %tsx[-5] ;
	call fib ;
	popv ;
	store %tsx[-6] %rvx ;
	store %tsx[-7] (+ %tsx[-4] %tsx[-6]) ;
	store %rvx %tsx[-7] ;
	popf 8 ;
	return ;
L16:
	noop ;
L14:
	noop ;
	popf 8 ;
	return ;
#  
# End of function fib
#  
L12:
	noop ;
	store t$0 0 ;
L17:
	store t$1 (<= t$0 12) ;
	jumpF t$1 L18 ;
	pushv t$0 ;
	call fib ;
	popv ;
	store t$2 %rvx ;
	print t$2 ;
	store t$3 (+ t$0 1) ;
	store t$0 t$3 ;
	jump L17 ;
L18:
	noop ;
	stop ;



## Testing parts of the compiler

In [12]:
from cuppa3_lex import lexer
from cuppa3_cc_frontend_gram import parser
from cuppa3_cc_state import state
from cuppa3_cc_tree_rewrite import walk as rewrite
from cuppa3_cc_codegen import walk as codegen
from cuppa3_cc_output import output
from grammar_stuff import dump_AST
from cuppa3_cc import cc
from pprint import pprint

In [13]:
state.initialize()

In [14]:
print(add)


declare add(a,b) 
{
    return a+b;
}

declare x = add(3,2);
put x;



In [15]:
parser.parse(add, lexer=lexer)

In [16]:
dump_AST(state.AST)


(seq 
  |(fundecl add 
  |  |(seq 
  |  |  |(id a) 
  |  |  |(seq 
  |  |  |  |(id b) 
  |  |  |  |(nil))) 
  |  |(block 
  |  |  |(seq 
  |  |  |  |(return 
  |  |  |  |  |(+ 
  |  |  |  |  |  |(id a) 
  |  |  |  |  |  |(id b))) 
  |  |  |  |(nil)))) 
  |(seq 
  |  |(declare x 
  |  |  |(callexp add 
  |  |  |  |(seq 
  |  |  |  |  |(integer 3) 
  |  |  |  |  |(seq 
  |  |  |  |  |  |(integer 2) 
  |  |  |  |  |  |(nil))))) 
  |  |(seq 
  |  |  |(put 
  |  |  |  |(id x)) 
  |  |  |(nil))))


In [17]:
t = rewrite(state.AST)
dump_AST(t)


(seq 
  |(fundef add 
  |  |(seq %tsx[0] 
  |  |  |(seq %tsx[-1] 
  |  |  |  |(nil))) 
  |  |(block 
  |  |  |(seq 
  |  |  |  |(return 
  |  |  |  |  |(+ %tsx[-2] 
  |  |  |  |  |  |(id %tsx[0]) 
  |  |  |  |  |  |(id %tsx[-1]))) 
  |  |  |  |(nil))) 3) 
  |(seq 
  |  |(assign t$1 
  |  |  |(callexp t$0 add 
  |  |  |  |(seq 
  |  |  |  |  |(integer 3) 
  |  |  |  |  |(seq 
  |  |  |  |  |  |(integer 2) 
  |  |  |  |  |  |(nil))))) 
  |  |(seq 
  |  |  |(put 
  |  |  |  |(id t$1)) 
  |  |  |(nil))))


In [18]:
print(output(codegen(t)))

	jump L19 ;
#  
# Start of function add
#  
add:
	pushf 3 ;
	store %tsx[0] %tsx[-4] ;
	store %tsx[-1] %tsx[-5] ;
	store %tsx[-2] (+ %tsx[0] %tsx[-1]) ;
	store %rvx %tsx[-2] ;
	popf 3 ;
	return ;
	popf 3 ;
	return ;
#  
# End of function add
#  
L19:
	noop ;
	pushv 2 ;
	pushv 3 ;
	call add ;
	popv ;
	popv ;
	store t$0 %rvx ;
	store t$1 t$0 ;
	print t$1 ;



In [19]:
factfunc = \
'''
// another recursive implementation of factorial
// no local variables other than the formal parameter
declare fact(x) 
{
     if (x <= 1)
        return 1;
     else 
        return fact(x-1) * x;
}

put fact(3)
'''
print(cc(factfunc))

	jump L20 ;
#  
# Start of function fact
#  
fact:
	pushf 5 ;
	store %tsx[0] %tsx[-6] ;
	store %tsx[-1] (<= %tsx[0] 1) ;
	jumpF %tsx[-1] L21 ;
	store %rvx 1 ;
	popf 5 ;
	return ;
	jump L22 ;
L21:
	store %tsx[-2] (- %tsx[0] 1) ;
	pushv %tsx[-2] ;
	call fact ;
	popv ;
	store %tsx[-3] %rvx ;
	store %tsx[-4] (* %tsx[-3] %tsx[0]) ;
	store %rvx %tsx[-4] ;
	popf 5 ;
	return ;
L22:
	noop ;
	popf 5 ;
	return ;
#  
# End of function fact
#  
L20:
	noop ;
	pushv 3 ;
	call fact ;
	popv ;
	store t$0 %rvx ;
	print t$0 ;
	stop ;



In [20]:
interp(cc(factfunc))

> 6
